# Quantitative Value Strategy

We are looking for stocks that are undervalue i.e. cheap relative to their business value (derived from earnings & assets). P/E ratio is used as metrics

Strategy will select 50 stocks with best value metrics and will give recommended trades.

# Library import

In [1]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

# Importing API token & List of stocks

In [2]:
stocks = pd.read_csv(r"C:\Users\nik10\iCloudDrive\Documents\Python Trading Robot\Project 1 - Equal Weight Index Fund\starter_files\sp_500_stocks.csv")
from secrets import IEX_CLOUD_API_TOKEN
IEX_CLOUD_API_TOKEN

'Tpk_059b97af715d417d9f49f50b51b1c448'

# Making Our First API Call

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()


# Obtaining data we need & parsing

In [4]:
data['peRatio']
price = data['latestPrice']
pe_ratio = data ['peRatio']

# Create dataframe for symbols & batch call

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker','Price', 'Price-to-Earnings Ratio','Number of shares to buy']

print(symbol_strings)

['A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA', 'CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD', 'GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
#    print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    #use split method to undo join
    for symbol in symbol_string.split(','):
    
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    data[symbol]['quote']['latestPrice'],
                    data[symbol]['quote']['peRatio'],
                    'N/A'
                ],
                index = my_columns),
        ignore_index = True    
        )

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of shares to buy
0,A,175.22,59.26,N/A
1,AAL,20.99,-2.09,N/A
2,AAP,209.64,22.08,N/A
3,AAPL,154.30,29.98,N/A
4,ABBV,124.30,33.03,N/A
...,...,...,...,...
500,YUM,135.40,31.21,N/A
501,ZBH,153.35,34.46,N/A
502,ZBRA,613.40,42.05,N/A
503,ZION,58.89,5.84,N/A


# Removing Glamour stocks
Glamour stock - a stock that is the opposite of the 'value' stock - highest price to earnings ratio. We want lowest PE ratio stocks but not negative

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio']> 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis =1, inplace = True)


In [8]:
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of shares to buy
0,EBAY,76.45,4.28,N/A
1,MPC,61.31,5.19,N/A
2,ZION,58.89,5.84,N/A
3,PRU,111.08,5.9,N/A
4,BIO,801.76,5.98,N/A
5,GM,49.90,6.02,N/A
6,COF,180.80,7.04,N/A
7,AFL,57.42,7.19,N/A
8,CINF,130.10,7.37,N/A
9,CE,163.96,7.4,N/A


# Calculating the number of shares to buy

In [9]:

def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')

    try:
        float(portfolio_size)
    #forces portfolio_size variable to become float
#    print (val)
    except ValueError:
        print('Only numerical characters are allowed! \nPlease try again:')
        portfolio_size = input('Enter the value of your portfolio:')

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [10]:
#portfolio_size
#output is a string
position_size = float(portfolio_size)/len(final_dataframe.index)
#   print(position_size)
#shows how much money should be invested in each stock

for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])

final_dataframe
#    print(math.floor(number_of_apple_shares)) #rounds down each position size

,Ticker,Price,Price-to-Earnings Ratio,Number of shares to buy,Number of Shares to Buy
0,EBAY,76.45,4.28,N/A,261.0
1,MPC,61.31,5.19,N/A,326.0
2,ZION,58.89,5.84,N/A,339.0
3,PRU,111.08,5.9,N/A,180.0
4,BIO,801.76,5.98,N/A,24.0
5,GM,49.90,6.02,N/A,400.0
6,COF,180.80,7.04,N/A,110.0
7,AFL,57.42,7.19,N/A,348.0
8,CINF,130.10,7.37,N/A,153.0
9,CE,163.96,7.4,N/A,121.0


# Building a better momentum strategy

Instead of relying on PE ratios, composite basket of valuation is typically used to improve quantitative value strategies. Following metrics are used:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA) 
* Enterprise Value divided by Gross Profit (EV/GP)

Some of the metrics need to be computed separately after pulling raw data. 

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
print(data)
# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprice_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprice_value/ebitda


# Enterprise Value divided by Gross Profit (EV/GP)

gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprice_value/gross_profit

{'AAPL': {'advanced-stats': {'beta': 1.5833366062053071, 'totalCash': 62884752461, 'currentDebt': 110179632095, 'revenue': 348366176068, 'grossProfit': 148990852216, 'totalRevenue': 353809730234, 'EBITDA': 114192289301, 'revenuePerShare': 21, 'revenuePerEmployee': 2369071.15, 'debtToEquity': 5.206129944730707, 'profitMargin': 0.2528115469039666, 'enterpriseValue': 2578902175361, 'enterpriseValueToRevenue': 7.3, 'priceToSales': 7.25, 'priceToBook': 39.32, 'forwardPERatio': 0.00238221787845079, 'pegRatio': 0.521711384955587, 'peHigh': 28.887777672714115, 'peLow': 14.825917607512, 'week52highDate': '2021-08-03', 'week52lowDate': '2020-09-21', 'putCallRatio': 0.455356764398298, 'companyName': 'Apple Inc', 'marketcap': 2514121446030, 'week52high': 155.46, 'week52low': 104.9, 'week52highSplitAdjustOnly': 152.52, 'week52highDateSplitAdjustOnly': '2021-08-04', 'week52lowSplitAdjustOnly': 108.1, 'week52lowDateSplitAdjustOnly': '2020-09-09', 'week52change': 0.1991337214392349, 'sharesOutstanding

Building DataFrame. `rv` stands for `robust value` which is the momentum score 

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-earnings ratio',
    'PE Percentile',
    'Price-to-book ratio',
    'PB Percentile',
    'Price-to-sales ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'

]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        
        enterprice_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
    
        try:
            ev_to_ebitda = enterprice_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
    
        try:
            ev_to_gross_profit = enterprice_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
    
        rv_dataframe = rv_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'                               
            ],
            index = rv_columns),
                    ignore_index = True    
        )
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,181.15,N/A,58.43,N/A,11.42,N/A,9.45,N/A,38.919317,N/A,17.543283,N/A,N/A
1,AAL,20.98,N/A,-2.1,N/A,-1.79,N/A,0.708,N/A,-4.628799,N/A,1.880783,N/A,N/A
2,AAP,211.58,N/A,22.72,N/A,3.93,N/A,1.18,N/A,11.074245,N/A,2.646299,N/A,N/A
3,AAPL,148.90,N/A,29.29,N/A,39.94,N/A,7.31,N/A,22.727251,N/A,17.716419,N/A,N/A
4,ABBV,121.10,N/A,33.83,N/A,17.19,N/A,4.09,N/A,11.394837,N/A,8.355706,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,134.10,N/A,31.43,N/A,-5.12,N/A,6.4,N/A,21.652179,N/A,10.479297,N/A,N/A
501,ZBH,151.45,N/A,34.23,N/A,2.5,N/A,4.07,N/A,20.601141,N/A,7.134164,N/A,N/A
502,ZBRA,590.04,N/A,41.81,N/A,12.24,N/A,6.34,N/A,32.252924,N/A,13.663107,N/A,N/A
503,ZION,59.45,N/A,6.05,N/A,1.28,N/A,2.7,N/A,4.754666,N/A,2.567800,N/A,N/A


# Dealing with Missing Data in Our DataFrame

Use pandas `isnull` to identify missing data:

In [13]:
#gives the missing data add .index for count
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,188.700,N/A,60.75,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,296.420,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,288.500,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,12.000,N/A,10.05,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,27.820,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.030,N/A,14.38,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
186,FLIR,59.680,N/A,32.68,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,34.910,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,3032.840,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
325,MXIM,107.310,N/A,34.04,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data can be done using two methods:
* Drop missing data (`dropna`)
* Replace missing data with a new value (`fillna`)

In [14]:
for column in ['Price-to-earnings ratio', 'Price-to-book ratio', 'Price-to-sales ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)
#we run a loop to fill missing value with mean values of the dataset

In [15]:
rv_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'Price-to-earnings ratio',
       'PE Percentile', 'Price-to-book ratio', 'PB Percentile',
       'Price-to-sales ratio', 'PS Percentile', 'EV/EBITDA',
       'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile', 'RV Score'],
      dtype='object')

# Calculating Value Percentiles
We calculate percentile for every stock
Following metrics are used:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

In [16]:
# 4:02:07
from scipy.stats import percentileofscore as score

metrics =  {'Price-to-earnings ratio':'PE Percentile',
    'Price-to-book ratio': 'PB Percentile',
    'Price-to-sales ratio':'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score( rv_dataframe[metric] , rv_dataframe.loc[row, metric])


rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,181.15,N/A,58.43,87.722772,11.42,81.782178,9.450,85.148515,38.919317,91.485149,17.543283,88.316832,N/A
1,AAL,20.98,N/A,-2.10,7.722772,-1.79,4.752475,0.708,6.336634,-4.628799,2.772277,1.880783,7.326733,N/A
2,AAP,211.58,N/A,22.72,41.782178,3.93,50.0,1.180,13.564356,11.074245,30.29703,2.646299,12.871287,N/A
3,AAPL,148.90,N/A,29.29,57.227723,39.94,96.237624,7.310,78.811881,22.727251,71.683168,17.716419,88.910891,N/A
4,ABBV,121.10,N/A,33.83,64.554455,17.19,90.09901,4.090,57.623762,11.394837,32.079208,8.355706,53.465347,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,134.10,N/A,31.43,60.594059,-5.12,4.356436,6.400,72.772277,21.652179,70.49505,10.479297,66.732673,N/A
501,ZBH,151.45,N/A,34.23,65.148515,2.50,31.881188,4.070,57.029703,20.601141,68.712871,7.134164,43.366337,N/A
502,ZBRA,590.04,N/A,41.81,76.336634,12.24,83.564356,6.340,72.079208,32.252924,87.524752,13.663107,78.811881,N/A
503,ZION,59.45,N/A,6.05,9.108911,1.28,11.683168,2.700,39.90099,4.754666,4.554455,2.567800,12.079208,N/A


# Calculating the RV score

In [17]:
from statistics import mean

for row in rv_dataframe.index:
        
        value_percentiles = []
        for metric in metrics.keys():
            value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
        rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,181.15,N/A,58.43,87.722772,11.42,81.782178,9.450,85.148515,38.919317,91.485149,17.543283,88.316832,86.891089
1,AAL,20.98,N/A,-2.10,7.722772,-1.79,4.752475,0.708,6.336634,-4.628799,2.772277,1.880783,7.326733,5.782178
2,AAP,211.58,N/A,22.72,41.782178,3.93,50.0,1.180,13.564356,11.074245,30.29703,2.646299,12.871287,29.70297
3,AAPL,148.90,N/A,29.29,57.227723,39.94,96.237624,7.310,78.811881,22.727251,71.683168,17.716419,88.910891,78.574257
4,ABBV,121.10,N/A,33.83,64.554455,17.19,90.09901,4.090,57.623762,11.394837,32.079208,8.355706,53.465347,59.564356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,134.10,N/A,31.43,60.594059,-5.12,4.356436,6.400,72.772277,21.652179,70.49505,10.479297,66.732673,54.990099
501,ZBH,151.45,N/A,34.23,65.148515,2.50,31.881188,4.070,57.029703,20.601141,68.712871,7.134164,43.366337,53.227723
502,ZBRA,590.04,N/A,41.81,76.336634,12.24,83.564356,6.340,72.079208,32.252924,87.524752,13.663107,78.811881,79.663366
503,ZION,59.45,N/A,6.05,9.108911,1.28,11.683168,2.700,39.90099,4.754666,4.554455,2.567800,12.079208,15.465347


# Selecting the 50 Best Value Stocks

In [19]:
rv_dataframe.sort_values('RV Score', ascending = True, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(inplace = True)
rv_dataframe.drop('index', axis =1, inplace = True)

rv_dataframe

D:\Users\nik10\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.21,N/A,8.20,11.287129,0.5060,4.950495,0.4252,2.772277,3.585892,3.762376,0.414363,0.990099,4.752475
1,PRU,110.70,N/A,6.00,8.712871,0.6896,5.346535,0.6497,5.346535,4.627746,4.356436,0.644860,1.386139,5.029703
2,AAL,20.98,N/A,-2.10,7.722772,-1.7900,4.752475,0.7080,6.336634,-4.628799,2.772277,1.880783,7.326733,5.782178
3,L,56.98,N/A,9.91,13.861386,0.8442,6.732673,0.9928,9.50495,5.348002,5.742574,0.954285,2.574257,7.683168
4,GM,51.30,N/A,6.01,8.910891,1.4000,14.257426,0.5323,3.564356,2.192061,3.168317,1.997768,8.712871,7.722772
5,MCK,209.26,N/A,-7.33,6.138614,-59.1000,1.188119,0.1323,0.792079,8.216205,17.425743,2.861284,14.257426,7.960396
6,BA,228.22,N/A,-15.12,4.752475,-8.1000,3.960396,2.1600,31.980198,-30.875085,0.792079,-209.847814,0.39604,8.376238
7,LNC,74.03,N/A,10.11,14.653465,0.6139,5.148515,0.6833,6.138614,7.700654,15.445545,0.682732,1.584158,8.594059
8,RE,275.05,N/A,8.06,11.089109,1.0300,8.316832,0.9900,9.108911,7.168746,12.277228,0.928005,2.376238,8.633663
9,MET,63.23,N/A,13.25,21.782178,0.8120,6.138614,0.7800,7.524752,6.060954,6.930693,0.796587,1.980198,8.871287


# Calculating the Number of Shares to Buy

In [20]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the value of your portfolio:')

    try:
        float(portfolio_size)
    #forces portfolio_size variable to become float
#    print (val)
    except ValueError:
        print('Only numerical characters are allowed! \nPlease try again:')
        portfolio_size = input('Enter the value of your portfolio:')

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [23]:
#portfolio_size
#output is a string
position_size = float(portfolio_size)/len(rv_dataframe.index)
#   print(position_size)
#shows how much money should be invested in each stock

for i in range(0, len(rv_dataframe.index)):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[i, 'Price'])

rv_dataframe
#    print(math.floor(number_of_apple_shares)) #rounds down each position size

D:\Users\nik10\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,Price-to-earnings ratio,PE Percentile,Price-to-book ratio,PB Percentile,Price-to-sales ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,27.21,735,8.20,11.287129,0.5060,4.950495,0.4252,2.772277,3.585892,3.762376,0.414363,0.990099,4.752475
1,PRU,110.70,180,6.00,8.712871,0.6896,5.346535,0.6497,5.346535,4.627746,4.356436,0.644860,1.386139,5.029703
2,AAL,20.98,953,-2.10,7.722772,-1.7900,4.752475,0.7080,6.336634,-4.628799,2.772277,1.880783,7.326733,5.782178
3,L,56.98,351,9.91,13.861386,0.8442,6.732673,0.9928,9.50495,5.348002,5.742574,0.954285,2.574257,7.683168
4,GM,51.30,389,6.01,8.910891,1.4000,14.257426,0.5323,3.564356,2.192061,3.168317,1.997768,8.712871,7.722772
5,MCK,209.26,95,-7.33,6.138614,-59.1000,1.188119,0.1323,0.792079,8.216205,17.425743,2.861284,14.257426,7.960396
6,BA,228.22,87,-15.12,4.752475,-8.1000,3.960396,2.1600,31.980198,-30.875085,0.792079,-209.847814,0.39604,8.376238
7,LNC,74.03,270,10.11,14.653465,0.6139,5.148515,0.6833,6.138614,7.700654,15.445545,0.682732,1.584158,8.594059
8,RE,275.05,72,8.06,11.089109,1.0300,8.316832,0.9900,9.108911,7.168746,12.277228,0.928005,2.376238,8.633663
9,MET,63.23,316,13.25,21.782178,0.8120,6.138614,0.7800,7.524752,6.060954,6.930693,0.796587,1.980198,8.871287


# Formatting Excel output

In [24]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

# Creating the Formats We'll Need For Our .xlsx File¶
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices 
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [25]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [26]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

# Save file

In [27]:
writer.save()